In [9]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt
import pickle
import os
import time

import seaborn as sns
sns.set()
from matplotlib import style
# style.use('ggplot')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
df = pd.read_csv('sp500_wti.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197718 entries, 0 to 1197717
Data columns (total 34 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Date                       1197718 non-null  datetime64[ns]
 1   Adj Close                  1197718 non-null  float64       
 2   volume_obv                 1197718 non-null  float64       
 3   volume_fi                  1197718 non-null  float64       
 4   volatility_bbm             1197718 non-null  float64       
 5   volatility_bbw             1197718 non-null  float64       
 6   volatility_bbhi            1197718 non-null  float64       
 7   volatility_bbli            1197718 non-null  float64       
 8   trend_macd                 1197718 non-null  float64       
 9   trend_macd_signal          1197718 non-null  float64       
 10  trend_macd_diff            1197718 non-null  float64       
 11  trend_psar_up_indicator    1197718 no

In [11]:
def cat_cont_split(df, maxcard=55, omit_vars=['Date', 'Adj Close', 'Return', 'target_return']):
    """Helper function that returns column names of categorical & continuous features from df."""
    cat_feats, cont_feats = [], []
    for col in df:
        if col in omit_vars: 
            continue
        if (df[col].dtype==int or df[col].dtype==float) and df[col].unique().shape[0] > maxcard:
            cont_feats.append(col)
        else:
            cat_feats.append(col)
    return cat_feats, cont_feats
    
cat_vars, cont_vars = cat_cont_split(df)
print(len(cat_vars), 'Categorical Features:')
print(cat_vars)
print(len(cont_vars), 'Continuous Features:')
print(cont_vars)

19 Categorical Features:
['volatility_bbhi', 'volatility_bbli', 'trend_psar_up_indicator', 'trend_psar_down_indicator', 'ticker', 'exchange', 'sector', 'industry', 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
11 Continuous Features:
['volume_obv', 'volume_fi', 'volatility_bbm', 'volatility_bbw', 'trend_macd', 'trend_macd_signal', 'trend_macd_diff', 'momentum_ao', 'momentum_roc', 'Dayofyear', 'Elapsed']


In [12]:
class Categorifier:
    ''' Transform categorical features into category types '''
    def apply_train(self, df, cat_vars):
        self.cat_vars = cat_vars
        self.categories = {}
        for v in self.cat_vars:
            df.loc[:, v] = df.loc[:, v].astype('category').cat.as_ordered()
            self.categories[v] = df[v].cat.categories
            
    def apply_test(self, df_test):
        for v in self.cat_vars:
            df_test.loc[:, v] = pd.Categorical(df[v], categories=self.categories[v], ordered=True)

cat = Categorifier()
cat.apply_train(df, cat_vars)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197718 entries, 0 to 1197717
Data columns (total 34 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Date                       1197718 non-null  datetime64[ns]
 1   Adj Close                  1197718 non-null  float64       
 2   volume_obv                 1197718 non-null  float64       
 3   volume_fi                  1197718 non-null  float64       
 4   volatility_bbm             1197718 non-null  float64       
 5   volatility_bbw             1197718 non-null  float64       
 6   volatility_bbhi            1197718 non-null  category      
 7   volatility_bbli            1197718 non-null  category      
 8   trend_macd                 1197718 non-null  float64       
 9   trend_macd_signal          1197718 non-null  float64       
 10  trend_macd_diff            1197718 non-null  float64       
 11  trend_psar_up_indicator    1197718 no

In [13]:
dep_var = 'target_return'
cont_vars = cont_vars + ['Return']
for c in ['ticker','exchange', 'sector', 'industry']:
    cat_vars.remove(c)
cat_sz = [(c, len(df[c].cat.categories)+1) for c in cat_vars]
# # emb_szs = [(nc, min(50, (nc+1)//2)) for _, nc in cat_sz]
emb_szs = [(nc, min(50, round(1.6*nc**0.56))) for _, nc in cat_sz]
emb_szs

[(3, 3),
 (3, 3),
 (3, 3),
 (3, 3),
 (12, 6),
 (13, 7),
 (54, 15),
 (32, 11),
 (6, 4),
 (3, 3),
 (3, 3),
 (3, 3),
 (3, 3),
 (3, 3),
 (2, 2)]

In [15]:
last_day = df['Date'].max()
cut_day = last_day + dt.timedelta(-90)
train_df = df[df['Date'] <= cut_day].sort_values(by='Date', ascending=False).reset_index(drop=True)
test_df = df[(df['Date'] > cut_day) & (df['Date'] < last_day)].sort_values(by='Date', ascending=False).reset_index(drop=True)
train_df.shape, test_df.shape, train_df.Date.min(), train_df.Date.max(), test_df.Date.min(), test_df.Date.max()

((1169210, 34),
 (28025, 34),
 Timestamp('2010-01-05 00:00:00'),
 Timestamp('2019-11-20 00:00:00'),
 Timestamp('2019-11-21 00:00:00'),
 Timestamp('2020-02-14 00:00:00'))

In [22]:
train_pred0 = pd.read_csv('./stage1Results/train_preds.csv', index_col='Date', parse_dates=True)
test_pred0 = pd.read_csv('./stage1Results/test_preds.csv', index_col='Date', parse_dates=True)

In [24]:
# df = train_df[train_df['ticker']=='AAPL']
df_pred.columns.to_list()

['A',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABMD',
 'ABT',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADS',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AGN',
 'AIG',
 'AIV',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'ALXN',
 'AMAT',
 'AMD',
 'AME',
 'AMG',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ARNC',
 'ATO',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BAX',
 'BBY',
 'BDX',
 'BEN',
 'BIIB',
 'BK',
 'BKNG',
 'BLK',
 'BLL',
 'BMY',
 'BR',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDNS',
 'CDW',
 'CE',
 'CELG',
 'CERN',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COG',
 'COO',
 'COP',
 'COST',
 'COTY',
 'CPB',
 'CPRT',
 'CRM',
 'CSCO',
 'CSX',
 'CTAS',
 'CTL',
 'CTSH',
 'CTXS',
 'CVS',
 'CVX'

In [33]:
from stage2_mod0 import *
ticker = 'AAPL'
train_ds = CrossDataset(train_df, ticker, train_pred0, dep_var, cat_vars, cont_vars)
mkts = train_ds.get_mkts()
train_dl = DataLoader(train_ds, 128, shuffle=True)
test_ds = CrossDataset(test_df, ticker, test_pred0, dep_var, cat_vars, cont_vars, mkts)

In [58]:
model = CrossModel(emb_szs, len(cont_vars), n_mkt=len(mkts), hidden_sizes=[1],
                    emb_dropout = 0.1, dropout_prob = [0.001, .01], use_bn=True)
print(model)

CrossModel(
  (embeds): ModuleList(
    (0): Embedding(3, 3)
    (1): Embedding(3, 3)
    (2): Embedding(3, 3)
    (3): Embedding(3, 3)
    (4): Embedding(12, 6)
    (5): Embedding(13, 7)
    (6): Embedding(54, 15)
    (7): Embedding(32, 11)
    (8): Embedding(6, 4)
    (9): Embedding(3, 3)
    (10): Embedding(3, 3)
    (11): Embedding(3, 3)
    (12): Embedding(3, 3)
    (13): Embedding(3, 3)
    (14): Embedding(2, 2)
  )
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (bn_cont): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn_mkt): BatchNorm1d(438, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mkt_fc): Linear(in_features=438, out_features=20, bias=True)
  (layers): Sequential(
    (0): Linear(in_features=104, out_features=1, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.001, inplace=False)
    (4): Linear(in_features=1, out_f

In [48]:
train_model(model, train_dl, n_epochs=2, lr=1e-3, weight_decay=0.2)
train_model(model, train_dl, n_epochs=2, lr=1e-4, weight_decay=0.2)

Epoch 1: training loss=4.918
Epoch 2: training loss=4.074
Epoch 1: training loss=4.020
Epoch 2: training loss=3.981


In [49]:
train_targets.shape, train_df.shape

((2488, 1), (1169210, 34))

In [46]:
t0 = time.time()
train_preds, train_targets = predict(model, train_ds, 1024)
print(time.time() - t0)
print(np.sum(train_targets[:, 0] != train_df[train_df.ticker==ticker]['target_return']))

100%|██████████| 3/3 [00:00<00:00, 57.37it/s]

0.05406999588012695
0


In [19]:
def MAE(y, y_hat):
    return np.mean(np.abs(y_hat-y))

def MAPE(y, y_hat):
    return np.mean(np.abs(y_hat-y) / (y+1e-3)) * 100

def RMSE(y, y_hat):
    return np.sqrt(np.mean((y_hat - y)**2))

def RMSPE(y, y_hat):
    return np.sqrt(np.mean(((y - y_hat) / (y+1e-3)) ** 2))

def scores(y, y_hat):
    return [MAE(y, y_hat), MAPE(y, y_hat), RMSE(y, y_hat), RMSPE(y, y_hat)]

In [20]:
train_dates = train_df.Date.unique()
test_dates = test_df.Date.unique()

is_sorted = lambda A : np.all(A[:-1] > A[1:])
is_sorted(train_dates), is_sorted(test_dates)

(True, True)

In [65]:
tickers = df.ticker.unique().tolist()
train_metrics, test_metrics = [], []
train_preds_df = pd.DataFrame(index=train_dates)
test_preds_df = pd.DataFrame(index=test_dates)
t0 = time.time()
for i, ticker in enumerate(tickers):
    if i > 20:
        break
    try:
        print("\n##### {} {}:".format(i, ticker))

        ## Prepare dataset & dataloader
        train_ds = CrossDataset(train_df, ticker, train_pred0, dep_var, cat_vars, cont_vars)
        mkts = train_ds.get_mkts()
        train_dl = DataLoader(train_ds, 128, shuffle=True)
        test_ds = CrossDataset(test_df, ticker, test_pred0, dep_var, cat_vars, cont_vars, mkts)

        ## Build & train model
        model = CrossModel(emb_szs, len(cont_vars), n_mkt=len(mkts), hidden_sizes=[1024, 512],
                        emb_dropout = 0.1, dropout_prob = [0.001, .01], use_bn=True) 
        train_model(model, train_dl, n_epochs=2, lr=1e-3, weight_decay=0.001)
        train_model(model, train_dl, n_epochs=2, lr=1e-4, weight_decay=0.001)

        ## Training Metrics
        yhat, y = predict(model, train_ds, 1024)

        temp = dict(zip(['MAE', 'MAPE', 'RMSE', 'RMSPE'], scores(y, yhat)))
        temp['ticker'] = ticker
        train_metrics.append(temp)

        idx = train_df[train_df.ticker==ticker].index
        dates = train_df.iloc[idx]['Date']
        df_temp = pd.DataFrame(yhat, index=dates, columns=[ticker])
        train_preds_df = train_preds_df.join(df_temp) 

        print("Training: ", temp)
    #     ## Test Metrics
    # #     idx = test_df[test_df.ticker==ticker].index
    # #     yhat, y = test_preds[idx], test_targets[idx]
    #     yhat, y = predict(model, test_ds)
    #     temp = dict(zip(['MAE', 'MAPE', 'RMSE', 'RMSPE'], scores(y, yhat)))
    #     temp['ticker'] = ticker
    #     test_metrics.append(temp)

    #     idx = test_df[test_df.ticker==ticker].index
    #     dates = test_df.iloc[idx]['Date']
    #     df_temp = pd.DataFrame(yhat, index=dates, columns=[ticker])
    #     test_preds_df = test_preds_df.join(df_temp) 

    #     print("Testing: ", temp)
        if i % 50 == 0:
            print("Elapsed: ", time.time() - t0)
    #     print('-'*20)
    except:
        pass

train_metrics = pd.DataFrame(train_metrics)
# test_metrics = pd.DataFrame(test_metrics)


##### 0 A:
Epoch 1: training loss=37.040
Epoch 2: training loss=36.064


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1: training loss=35.799
Epoch 2: training loss=35.712


100%|██████████| 3/3 [00:00<00:00, 67.56it/s]


Training:  {'MAE': 1.2198000209096787, 'MAPE': 183.7471269881054, 'RMSE': 1.7105522564341216, 'RMSPE': 28.701082007990756, 'ticker': 'A'}
Elapsed:  0.8015999794006348

##### 1 AAL:
Epoch 1: training loss=42.849
Epoch 2: training loss=40.894


...

Training:  {'MAE': 1.2044075972951844, 'MAPE': 193.20011210979138, 'RMSE': 1.696626727430554, 'RMSPE': 31.83230173488211, 'ticker': 'AGN'}

##### 20 AIG:
Epoch 1: training loss=37.781
Epoch 2: training loss=36.782


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1: training loss=36.412
Epoch 2: training loss=36.283


100%|██████████| 3/3 [00:00<00:00, 65.91it/s]

Training:  {'MAE': 1.262444630202348, 'MAPE': 173.3078068202637, 'RMSE': 1.8691512189583812, 'RMSPE': 40.44557160983009, 'ticker': 'AIG'}


In [48]:
codes.shape, conts.shape, 

((2488, 14), (2488, 12))

In [23]:
df.head()

,Date,Adj Close,volume_obv,volume_fi,volatility_bbm,volatility_bbw,volatility_bbhi,volatility_bbli,trend_macd,trend_macd_signal,trend_macd_diff,trend_psar_up_indicator,trend_psar_down_indicator,momentum_ao,momentum_roc,Return,target_return,ticker,exchange,sector,industry,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
417,2019-11-20,9.893816,29.976614,0.002931,8.398610,1.036641,0.0,0.0,0.264809,0.274550,-0.009741,0.0,0.0,0.590641,2.209710,-1.164147,-0.448343,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,47,20,2,324,False,False,False,False,False,False,1574208000
556,2019-11-19,10.010351,30.191781,0.007056,8.365885,1.086402,0.0,0.0,0.279793,0.276985,0.002808,0.0,0.0,0.632203,4.092722,-0.303256,-1.164147,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,47,19,1,323,False,False,False,False,False,False,1574121600
1420,2019-11-18,10.040800,30.346050,0.008913,8.322824,1.126840,0.0,0.0,0.284524,0.276283,0.008241,0.0,0.0,0.644790,7.372573,0.504213,-0.303256,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,47,18,0,322,False,False,False,False,False,False,1574035200
1715,2019-11-15,9.990427,30.170441,0.009116,8.279338,1.126492,0.0,0.0,0.283727,0.274223,0.009504,0.0,0.0,0.647212,9.249369,1.187936,0.504213,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,46,15,4,319,False,False,False,False,False,False,1573776000
2004,2019-11-14,9.873140,29.967483,0.007183,8.231337,1.155210,0.0,0.0,0.283045,0.271847,0.011198,0.0,0.0,0.649719,7.953480,-0.691945,1.187936,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,46,14,3,318,False,False,False,False,False,False,1573689600


In [32]:
# pred_df = pd.read_csv('./stage1Results/train_preds.csv', parse_dates=True)
pred_df = pd.read_csv('./stage1Results/train_preds.csv', index_col='Date', parse_dates=True)
pred_df.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AFL,AGN,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMD,AME,AMG,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ARNC,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BIIB,BK,BKNG,BLK,BLL,BMY,BR,BSX,BWA,BXP,C,CAG,CAH,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CELG,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTXS,CVS,CVX,CXO,D,DAL,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DRE,DRI,DTE,DUK,DVA,DVN,DXC,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOG,HOLX,HON,HP,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,IP,IPG,IPGP,IQV,IR,IRM,ISRG,IT,ITW,IVZ,JBHT,JCI,JEC,JKHY,JNJ,JNPR,JPM,JWN,K,KEY,KEYS,KHC,KIM,KLAC,KMB,KMI,KMX,KO,KR,KSS,KSU,L,LB,LDOS,LEG,LEN,LH,LKQ,LLY,LMT,LNC,LNT,LOW,LRCX,LUV,LW,LYB,M,MA,MAA,MAC,MAR,MAS,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MKTX,MLM,MMC,MMM,MNST,MO,MOS,MPC,MRK,MRO,MS,MSCI,MSFT,MSI,MTB,MTD,MU,MXIM,MYL,NBL,NCLH,NDAQ,NEE,NEM,NFLX,NI,NKE,NKTR,NLSN,NOC,NOV,NRG,NSC,NTAP,NTRS,NUE,NVDA,NVR,NWL,NWS,NWSA,O,OKE,OMC,ORCL,ORLY,OXY,PAYX,PBCT,PCAR,PEG,PEP,PFE,PFG,PG,PGR,PH,PHM,PKG,PKI,PLD,PM,PNC,PNR,PNW,PPG,PPL,PRGO,PRU,PSA,PSX,PVH,PWR,PXD,PYPL,QCOM,QRVO,RCL,RE,REG,REGN,RF,RHI,RJF,RL,RMD,ROK,ROL,ROP,ROST,RSG,RTN,SBAC,SBUX,SCHW,SEE,SHW,SIVB,SJM,SLB,SLG,SNA,SNPS,SO,SPG,SPGI,SRE,STT,STX,STZ,SWK,SWKS,SYF,SYK,SYY,T,TAP,TDG,TEL,TFX,TGT,TIF,TJX,TMO,TMUS,TPR,TRIP,TROW,TRV,TSCO,TSN,TTWO,TWTR,TXN,TXT,UA,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,UTX,V,VAR,VFC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRK,WU,WY,WYNN,XEC,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-20,-0.173125,-0.915990,-1.266274,-0.563448,-0.505165,-0.475396,-0.170284,-1.061611,0.031242,-0.464184,-0.354372,0.175016,-1.044149,0.371353,-1.039570,-0.039043,0.138851,-0.063987,-1.103722,-0.244323,-0.709359,0.031519,-0.608879,-0.493599,0.206911,-0.467285,-0.508342,-1.474490,-0.469405,-0.853380,-0.379162,-1.108141,-0.755342,-0.725681,-0.694651,-0.776903,-0.693320,-0.438066,0.631095,-1.189334,-1.162159,-0.500861,-0.641704,-0.142402,-0.980200,-0.831829,0.720244,-0.319645,-0.455295,-0.781324,-0.248025,-0.401843,0.347991,-0.712499,-0.579940,-0.559700,-0.271509,-0.373694,-0.246048,-0.763364,-1.015478,0.309072,-1.046129,-0.656266,-0.631902,-1.170471,-0.023778,-0.443180,-0.647076,-0.339136,-0.080724,-0.137607,-0.217074,-0.604623,-0.784834,-0.424104,-0.490571,0.231049,-1.032536,-0.694422,-0.090471,-0.100777,0.358576,-0.233840,-0.297658,-0.377878,-0.799246,-0.717322,-0.075905,-1.420799,-1.607565,-0.021323,-0.303270,-0.285880,-0.667543,-0.926242,0.002908,-0.637585,-0.390405,-0.716299,-0.091550,-0.019328,-0.100373,-0.638572,-0.020921,-0.841599,-0.859664,-0.747941,-0.065412,-0.430977,-0.700090,-0.395895,-0.093961,-0.348624,-1.002043,0.277906,-0.627961,-0.038071,-1.011249,-0.314382,-0.220611,0.138808,-0.730754,-0.711230,-0.109195,-0.411863,-0.781826,-0.540328,-0.356528,0.249221,-0.367685,0.114127,-0.184881,-0.209127,-0.219563,0.152647,0.529088,-0.165034,-0.377058,-0.425565,-0.434033,0.504307,-0.231134,-0.867934,-0.833577,-0.843833,0.005310,-0.816996,0.399361,-0.335992,0.166722,-0.64851

In [31]:
pred_df.columns

Index(['Date', 'A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN',
       ...
       'XEL', 'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YUM', 'ZBH', 'ZION',
       'ZTS'],
      dtype='object', length=486)

In [40]:
pred_df.iloc[pred_df.index==df.Date].head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AFL,AGN,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMD,AME,AMG,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ARNC,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BIIB,BK,BKNG,BLK,BLL,BMY,BR,BSX,BWA,BXP,C,CAG,CAH,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CELG,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTXS,CVS,CVX,CXO,D,DAL,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DRE,DRI,DTE,DUK,DVA,DVN,DXC,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOG,HOLX,HON,HP,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,IP,IPG,IPGP,IQV,IR,IRM,ISRG,IT,ITW,IVZ,JBHT,JCI,JEC,JKHY,JNJ,JNPR,JPM,JWN,K,KEY,KEYS,KHC,KIM,KLAC,KMB,KMI,KMX,KO,KR,KSS,KSU,L,LB,LDOS,LEG,LEN,LH,LKQ,LLY,LMT,LNC,LNT,LOW,LRCX,LUV,LW,LYB,M,MA,MAA,MAC,MAR,MAS,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MKTX,MLM,MMC,MMM,MNST,MO,MOS,MPC,MRK,MRO,MS,MSCI,MSFT,MSI,MTB,MTD,MU,MXIM,MYL,NBL,NCLH,NDAQ,NEE,NEM,NFLX,NI,NKE,NKTR,NLSN,NOC,NOV,NRG,NSC,NTAP,NTRS,NUE,NVDA,NVR,NWL,NWS,NWSA,O,OKE,OMC,ORCL,ORLY,OXY,PAYX,PBCT,PCAR,PEG,PEP,PFE,PFG,PG,PGR,PH,PHM,PKG,PKI,PLD,PM,PNC,PNR,PNW,PPG,PPL,PRGO,PRU,PSA,PSX,PVH,PWR,PXD,PYPL,QCOM,QRVO,RCL,RE,REG,REGN,RF,RHI,RJF,RL,RMD,ROK,ROL,ROP,ROST,RSG,RTN,SBAC,SBUX,SCHW,SEE,SHW,SIVB,SJM,SLB,SLG,SNA,SNPS,SO,SPG,SPGI,SRE,STT,STX,STZ,SWK,SWKS,SYF,SYK,SYY,T,TAP,TDG,TEL,TFX,TGT,TIF,TJX,TMO,TMUS,TPR,TRIP,TROW,TRV,TSCO,TSN,TTWO,TWTR,TXN,TXT,UA,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,UTX,V,VAR,VFC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRK,WU,WY,WYNN,XEC,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-20,-0.173125,-0.915990,-1.266274,-0.563448,-0.505165,-0.475396,-0.170284,-1.061611,0.031242,-0.464184,-0.354372,0.175016,-1.044149,0.371353,-1.039570,-0.039043,0.138851,-0.063987,-1.103722,-0.244323,-0.709359,0.031519,-0.608879,-0.493599,0.206911,-0.467285,-0.508342,-1.474490,-0.469405,-0.853380,-0.379162,-1.108141,-0.755342,-0.725681,-0.694651,-0.776903,-0.693320,-0.438066,0.631095,-1.189334,-1.162159,-0.500861,-0.641704,-0.142402,-0.980200,-0.831829,0.720244,-0.319645,-0.455295,-0.781324,-0.248025,-0.401843,0.347991,-0.712499,-0.579940,-0.559700,-0.271509,-0.373694,-0.246048,-0.763364,-1.015478,0.309072,-1.046129,-0.656266,-0.631902,-1.170471,-0.023778,-0.443180,-0.647076,-0.339136,-0.080724,-0.137607,-0.217074,-0.604623,-0.784834,-0.424104,-0.490571,0.231049,-1.032536,-0.694422,-0.090471,-0.100777,0.358576,-0.233840,-0.297658,-0.377878,-0.799246,-0.717322,-0.075905,-1.420799,-1.607565,-0.021323,-0.303270,-0.285880,-0.667543,-0.926242,0.002908,-0.637585,-0.390405,-0.716299,-0.091550,-0.019328,-0.100373,-0.638572,-0.020921,-0.841599,-0.859664,-0.747941,-0.065412,-0.430977,-0.700090,-0.395895,-0.093961,-0.348624,-1.002043,0.277906,-0.627961,-0.038071,-1.011249,-0.314382,-0.220611,0.138808,-0.730754,-0.711230,-0.109195,-0.411863,-0.781826,-0.540328,-0.356528,0.249221,-0.367685,0.114127,-0.184881,-0.209127,-0.219563,0.152647,0.529088,-0.165034,-0.377058,-0.425565,-0.434033,0.504307,-0.231134,-0.867934,-0.833577,-0.843833,0.005310,-0.816996,0.399361,-0.335992,0.166722,-0.64851

In [41]:
df.head()

,Date,Adj Close,volume_obv,volume_fi,volatility_bbm,volatility_bbw,volatility_bbhi,volatility_bbli,trend_macd,trend_macd_signal,trend_macd_diff,trend_psar_up_indicator,trend_psar_down_indicator,momentum_ao,momentum_roc,Return,target_return,ticker,exchange,sector,industry,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
417,2019-11-20,9.893816,29.976614,0.002931,8.398610,1.036641,0.0,0.0,0.264809,0.274550,-0.009741,0.0,0.0,0.590641,2.209710,-1.164147,-0.448343,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,47,20,2,324,False,False,False,False,False,False,1574208000
556,2019-11-19,10.010351,30.191781,0.007056,8.365885,1.086402,0.0,0.0,0.279793,0.276985,0.002808,0.0,0.0,0.632203,4.092722,-0.303256,-1.164147,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,47,19,1,323,False,False,False,False,False,False,1574121600
1420,2019-11-18,10.040800,30.346050,0.008913,8.322824,1.126840,0.0,0.0,0.284524,0.276283,0.008241,0.0,0.0,0.644790,7.372573,0.504213,-0.303256,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,47,18,0,322,False,False,False,False,False,False,1574035200
1715,2019-11-15,9.990427,30.170441,0.009116,8.279338,1.126492,0.0,0.0,0.283727,0.274223,0.009504,0.0,0.0,0.647212,9.249369,1.187936,0.504213,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,46,15,4,319,False,False,False,False,False,False,1573776000
2004,2019-11-14,9.873140,29.967483,0.007183,8.231337,1.155210,0.0,0.0,0.283045,0.271847,0.011198,0.0,0.0,0.649719,7.953480,-0.691945,1.187936,AAPL,NASDAQ,TECHNOLOGY,COMPUTER MANUFACTURING,2019,11,46,14,3,318,False,False,False,False,False,False,1573689600


In [42]:
pred_df.iloc[pred_df.index==df.Date].shape

(2488, 485)

In [46]:
temp_df = pred_df.iloc[pred_df.index==df.Date].dropna(axis=1)

In [47]:
preds = [content.values for col_name, content in temp_df.items()]
preds

[array([-0.17312539,  0.01402778, -1.8871604 , ...,  0.33693415,
         0.44894734,  1.0031636 ]),
 array([-0.9159904 , -1.2484003 , -1.8055273 , ...,  0.15354112,
         1.1708678 ,  0.28679487]),
 array([-1.2662735 ,  0.04114997, -1.3805407 , ...,  1.0169462 ,
         0.34665558,  1.5469205 ]),
 array([-0.56344754,  0.05968824, -0.99422777, ...,  0.22380003,
        -0.01422124,  0.6806732 ]),
 array([-0.4753958 ,  0.26632422, -1.151644  , ...,  0.37918556,
         0.04393085,  0.6995535 ]),
 array([-0.17028378, -0.64960563,  1.4545381 , ...,  0.42648703,
         0.11855476,  0.5379782 ]),
 array([-1.0616109 , -0.04810709, -1.003759  , ...,  0.0309002 ,
         0.48805013,  0.83643854]),
 array([ 0.03124174,  0.32552484, -1.805057  , ...,  0.2594015 ,
         0.15754615,  0.0400996 ]),
 array([-0.46418437,  0.02081237, -1.0429325 , ...,  0.17911068,
         0.12461367,  0.48632082]),
 array([-0.3543724 ,  0.02774569, -1.3917005 , ...,  0.13659166,
         0.00294465,  1.03